# 3a. Enrichment Table
  --------------------------------------------------------------------

Create the enrichment table that contains a lookup of postcode and returns a socioeconomic index (`socioeconomic_idx`). The enriched stream contains the original data and the enriched data.

This demonstrates a simple way of using V3IO Key-value (KV) to store lookup data.

The data is stored to `/User/examples/model-deployment-with-streaming/data/enrichment-table`. You can create similar enrichments using the same method.

![Model deployment with streaming Real-time operational Pipeline](../../assets/images/model-deployment-with-streaming.png)

## Initialize

Load the project

In [1]:
from mlrun import load_project
from os import path

project_path = path.abspath('conf')
project = load_project(project_path)

In [2]:
container = project.params.get('CONTAINER')
enrichment_table_path = project.params.get('ENRICHMENT_TABLE_PATH')
print(f'Container: {container}\nEnrichment table path: {enrichment_table_path}')

Container: users
Enrichment table path: iguazio/examples/model-deployment-with-streaming/data/enrichment-table


### Create V3IO Client

With the dataplane client you can manipulate data in the platform's multi-model data layer, including:
* Objects
* Key-values (NoSQL)
* Streams
* Containers

Under the hood, the client connects through the platform's web API (https://www.iguazio.com/docs/reference/latest-release/api-reference/web-apis/) and wraps each low level API with an interface. Calls are blocking, but you can use the batching interface to send multiple requests in parallel for greater performance. 

In [3]:
import v3io.dataplane
from os import getenv
v3io_client = v3io.dataplane.Client(endpoint=project.params.get('WEB_API'),
                                    access_key=getenv('V3IO_ACCESS_KEY'))

### Create enrichment table
____________________________

We will create enrichment table where the key is postal-code and the value is the socioeconomic index at the area.

To get the highest possible throughput, we can send many requests towards the data layer and wait for all the responses to arrive (rather than send each request and wait for the response). The SDK supports this through batching. Any API call can be made through the client's built in `batch` object. The API call receives the exact same arguments it would normally receive (except for `raise_for_status`), and does not block until the response arrives. To wait for all pending responses, call `wait()` on the `batch` object:

In [4]:
from random import randint
for postcode in range(10000,99999):
    remainder = postcode % 3
    if remainder == 0:
        idx = randint(3,5)
    elif remainder == 1:
        idx = randint(1,3)
    else:
        idx = randint(5,7)

    attr = {'postcode': postcode ,'socioeconomic_idx': idx}
    v3io_client.batch.put_item(container=container,
                               path=path.join(enrichment_table_path, str(postcode)),
                               attributes=attr)

responses = v3io_client.batch.wait()

The looped `put_item` interface above will send all `put item` requests to the data layer in parallel. When `wait` is called, it will block until either all responses arrive (in which case it will return a `Responses` object, containing the `responses` of each call) or an error occurs - in which case an exception is thrown. You can pass `raise_for_status` to `wait`, and it behaves as explained above.

> Note: The `batch` object is stateful, so you can only create one batch at a time. However, you can create multiple parallel batches yourself through the client's `create_batch()` interface

## Done

Continue to [**3b-enrich-stream.ipynb**](3b-enrich-stream.ipynb) to enrich the stream using the enrichment table.